In [352]:
import random
random.seed(1) # pros resultados serem consistentes

import logging; logging.basicConfig(level=logging.INFO)
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import logictensornetworks as ltn

plt.rcParams['font.size'] = 12
plt.rcParams['axes.linewidth'] = 1

In [353]:
import nltk
nltk.download('mac_morpho')

[nltk_data] Downloading package mac_morpho to
[nltk_data]     /home/castro/nltk_data...
[nltk_data]   Package mac_morpho is already up-to-date!


True

In [354]:
import gensim
import os
from nltk.corpus import mac_morpho
import glob
path = ['./false_text/','./true_text/']

model = None
if(not os.path.isfile('words.embedding')) :
    model = gensim.models.Word2Vec(mac_morpho.sents())
    new_texts = []
    for i in range(2):
        for file_name in glob.glob(path[i]+"*.txt"):
            with open(file_name, 'r') as file:  
                text = list(gensim.utils.tokenize(file.read()))
                new_texts.append(text)
    model.min_count = 1
    model.build_vocab(new_texts, update=True)
    model.train(new_texts, total_examples=len(new_texts), epochs=model.epochs)
    
    model.save('words.embedding')
else:
    model = gensim.models.Word2Vec.load('words.embedding')

INFO:gensim.utils:loading Word2Vec object from words.embedding
INFO:gensim.utils:loading wv recursively from words.embedding.wv.* with mmap=None
INFO:gensim.utils:setting ignored attribute cum_table to None
INFO:gensim.utils:Word2Vec lifecycle event {'fname': 'words.embedding', 'datetime': '2021-12-03T21:21:40.130206', 'gensim': '4.1.2', 'python': '3.8.10 (default, Sep 28 2021, 16:10:42) \n[GCC 9.3.0]', 'platform': 'Linux-5.10.60.1-microsoft-standard-WSL2-x86_64-with-glibc2.29', 'event': 'loaded'}


In [355]:
data = []
labels = []
qtd_words = 10
null_word = [0]*100
big_word_count = 0
for i in range(2):
    for file_name in glob.glob(path[i]+"*.txt"):
        with open(file_name, 'r') as file:
            text = list(gensim.utils.tokenize(file.read()))
            vec_words = [model.wv[word] for word in text]
            data.append(vec_words)
            labels.append(i>0)
            
            big_word_count = max(big_word_count, len(text))
len(data)

100

In [356]:

# same dimensions
for i, val in enumerate(data):
    need = big_word_count - len(val)
    data[i].extend(np.array([[1.0]*100]*need, dtype=np.float32))

# shuffle
tmp = list(zip(data, labels))
random.shuffle(tmp)
data, labels = zip(*tmp)

# to numpy array
data = np.array(data)
labels = np.array(labels)

In [357]:
import math

nr_samples_train = math.ceil(0.7*len(data))
batch_size = 32
print(nr_samples_train, batch_size)
ds_train = tf.data.Dataset\
        .from_tensor_slices((data[:nr_samples_train], labels[:nr_samples_train]))\
        .batch(batch_size)
ds_test = tf.data.Dataset\
        .from_tensor_slices((data[nr_samples_train:], labels[nr_samples_train:]))\
        .batch(batch_size)


70 32


In [358]:
Not = ltn.Wrapper_Connective(ltn.fuzzy_ops.Not_Std())
And = ltn.Wrapper_Connective(ltn.fuzzy_ops.And_Prod())
Or = ltn.Wrapper_Connective(ltn.fuzzy_ops.Or_ProbSum())
Implies = ltn.Wrapper_Connective(ltn.fuzzy_ops.Implies_Reichenbach())
Forall = ltn.Wrapper_Quantifier(ltn.fuzzy_ops.Aggreg_pMeanError(p=2),semantics="forall")
Exists = ltn.Wrapper_Quantifier(ltn.fuzzy_ops.Aggreg_pMean(p=2),semantics="exists")
formula_aggregator = ltn.Wrapper_Formula_Aggregator(ltn.fuzzy_ops.Aggreg_pMeanError(p=2))

In [359]:
Valid = ltn.Predicate.MLP([(big_word_count, 100)],hidden_layer_sizes=(16,16))

In [360]:
@tf.function
def axioms(features, labels):
    Valid_ex = ltn.Variable("Valid_ex", features[labels])
    Invalid_ex  = ltn.Variable("Invalid_ex", features[tf.logical_not(labels)])
    axioms = [
        Forall(Valid_ex, Valid(Valid_ex)),
        Forall(Invalid_ex, Not(Valid(Invalid_ex)))
    ]
    sat_level = formula_aggregator(axioms).tensor
    return sat_level

In [361]:
mean_metrics = tf.keras.metrics.Mean()

trainable_variables = Valid.trainable_variables
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0000005)
test_metrics = {}
train_metrics = {}
for epoch in range(10000):
    for _data, _labels in ds_train:
        with tf.GradientTape() as tape:
            loss = 1. - axioms(_data, _labels)
        grads = tape.gradient(loss, trainable_variables)
        optimizer.apply_gradients(zip(grads, trainable_variables))
    if epoch%500 == 0:
        mean_metrics.reset_states()
        for _data, _labels in ds_test:
            mean_metrics(axioms(_data, _labels))

        test_metrics[epoch] = mean_metrics.result()
        print("Epoch %d: Sat Level Test %.3f"%(epoch, mean_metrics.result() ))

        mean_metrics.reset_states()
        for _data, _labels in ds_train:
            mean_metrics(axioms(_data, _labels))
            
        train_metrics[epoch] = mean_metrics.result()
        print("Epoch %d: Sat Level Train %.3f"%(epoch, mean_metrics.result() ))   

mean_metrics.reset_states()
for _data, _labels in ds_test:
    mean_metrics(axioms(_data, _labels))
print("Training finished at Epoch %d with Sat Level %.3f"%(epoch, mean_metrics.result() ))

Epoch 0: Sat Level Test 0.492
Epoch 0: Sat Level Train 0.502
Epoch 500: Sat Level Test 0.518
Epoch 500: Sat Level Train 0.594
Epoch 1000: Sat Level Test 0.526
Epoch 1000: Sat Level Train 0.636
Epoch 1500: Sat Level Test 0.531
Epoch 1500: Sat Level Train 0.671
Epoch 2000: Sat Level Test 0.534
Epoch 2000: Sat Level Train 0.705
Epoch 2500: Sat Level Test 0.535
Epoch 2500: Sat Level Train 0.735
Epoch 3000: Sat Level Test 0.539
Epoch 3000: Sat Level Train 0.760
Epoch 3500: Sat Level Test 0.542
Epoch 3500: Sat Level Train 0.784
Epoch 4000: Sat Level Test 0.543
Epoch 4000: Sat Level Train 0.805
Epoch 4500: Sat Level Test 0.542
Epoch 4500: Sat Level Train 0.823
Epoch 5000: Sat Level Test 0.542
Epoch 5000: Sat Level Train 0.841
Epoch 5500: Sat Level Test 0.541
Epoch 5500: Sat Level Train 0.857
Epoch 6000: Sat Level Test 0.541
Epoch 6000: Sat Level Train 0.873
Epoch 6500: Sat Level Test 0.540
Epoch 6500: Sat Level Train 0.888
Epoch 7000: Sat Level Test 0.538
Epoch 7000: Sat Level Train 0.901
Epo

In [364]:
print(train_metrics)
print(test_metrics)

{0: <tf.Tensor: shape=(), dtype=float32, numpy=0.5020493>, 500: <tf.Tensor: shape=(), dtype=float32, numpy=0.59402853>, 1000: <tf.Tensor: shape=(), dtype=float32, numpy=0.6355082>, 1500: <tf.Tensor: shape=(), dtype=float32, numpy=0.67119044>, 2000: <tf.Tensor: shape=(), dtype=float32, numpy=0.70526916>, 2500: <tf.Tensor: shape=(), dtype=float32, numpy=0.7345576>, 3000: <tf.Tensor: shape=(), dtype=float32, numpy=0.76046085>, 3500: <tf.Tensor: shape=(), dtype=float32, numpy=0.7839164>, 4000: <tf.Tensor: shape=(), dtype=float32, numpy=0.8046182>, 4500: <tf.Tensor: shape=(), dtype=float32, numpy=0.8233612>, 5000: <tf.Tensor: shape=(), dtype=float32, numpy=0.84083456>, 5500: <tf.Tensor: shape=(), dtype=float32, numpy=0.8573732>, 6000: <tf.Tensor: shape=(), dtype=float32, numpy=0.87300014>, 6500: <tf.Tensor: shape=(), dtype=float32, numpy=0.8878417>, 7000: <tf.Tensor: shape=(), dtype=float32, numpy=0.901402>, 7500: <tf.Tensor: shape=(), dtype=float32, numpy=0.9137239>, 8000: <tf.Tensor: shap